In [ ]:
!pip install pandas

In [ ]:
import wget, os

# Getting the name of the Spark Connector from its URL
dataset = url.split("/")[-1]

# Download it only if it's not present
if dataset not in os.listdir():
    print("Downlading dataset")
    dataset = wget.download(url)

In [ ]:
url = 'https://github.com/neo4j-contrib/neo4j-spark-connector/releases/download/4.1.5/neo4j-connector-apache-spark_2.12-4.1.5_for_spark_3.jar'

# Getting the name of the Spark Connector from its URL
filename = url.split("/")[-1]

# Download it only if it's not present
if filename not in os.listdir():
    print("Downlading Spark Connector")
    filename = wget.download(url)

In [ ]:
from pyspark.sql import SparkSession
import os
import pandas as pd

# Providing the downloaded jar to the pyspark-shell
os.environ[
    'PYSPARK_SUBMIT_ARGS'] = f'--jars {filename} pyspark-shell'

#Ip address of the Neo4j database
NEO4J_IP_ADDRESS = "linktodb"

# Building our SparkSession
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1536m").\
        getOrCreate()

In [ ]:
df_pd = pd.read_csv(dataset)
sparkDF=spark.createDataFrame(df_pd)
sparkDF.show()

In [ ]:
#TODO: define create for this use case
sparkDF.write.format("org.neo4j.spark.DataSource")\
.mode("overwrite")\
.option("url", f"bolt://{NEO4J_IP_ADDRESS}:7687")\
.option("authentication.basic.username", "neo4j")\
.option("authentication.basic.password", "test")\
.option("batch.size", "5000")\
.option("database", "neo4j")\
.option("labels","Test")\
.option("node.keys","Postcode")\
.option("script","""CREATE CONSTRAINT IF NOT EXISTS FOR (t:Test) REQUIRE t.Postcode IS UNIQUE;""")\
.save()

In [ ]:
# Close the SparkSession
spark.stop()